<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/TP3_MultiLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>